In [25]:
! pip install kaggle

In [26]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json 

The syntax of the command is incorrect.
'cp' is not recognized as an internal or external command,
operable program or batch file.
'chmod' is not recognized as an internal or external command,
operable program or batch file.


In [27]:
import os
import shutil

# Create the .kaggle directory in the user's home directory
kaggle_dir = os.path.join(os.path.expanduser("~"), ".kaggle")
if not os.path.exists(kaggle_dir):
    os.makedirs(kaggle_dir)

# Define the path to the kaggle.json file
kaggle_json_path = "C:/Users/Yashr/.kaggle/kaggle.json"


print("kaggle.json file configured successfully.")

kaggle.json file configured successfully.


In [28]:
#API to fetch the dataset from kaggle
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


In [29]:
#extracting the compressed dataset

from zipfile import ZipFile
dataset = 'C:/Users/Yashr/OneDrive/Desktop/python project/sentiment140.zip'

with ZipFile(dataset,'r') as zips:
    zips.extractall()
    print('the dataset is extracted')

the dataset is extracted


In [30]:
#import the required dependencies
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [31]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Yashr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

data processing


In [32]:
#we have to add encoding= iso-8859-1
df=pd.read_csv('training.1600000.processed.noemoticon.csv',encoding='iso-8859-1')

In [33]:
df.shape

(1599999, 6)

In [34]:
df.head(5)

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [35]:
# naming the columns and reading the dataset again

columns_names = ['target','id','date','flag','user','text']
df=pd.read_csv('training.1600000.processed.noemoticon.csv',names = columns_names,encoding='iso-8859-1')

In [36]:
df.shape

(1600000, 6)

In [37]:
df.sample(5)

,target,id,date,flag,user,text
197258,0,1970964845,Sat May 30 05:48:31 PDT 2009,NO_QUERY,adriwadri,aww.time for me to go off i had fun with @Ste...
1242358,4,1994227198,Mon Jun 01 11:36:34 PDT 2009,NO_QUERY,ChristySeason,@jsandford I can't wait! I've been to 5 No Dou...
234296,0,1979556143,Sun May 31 03:33:19 PDT 2009,NO_QUERY,fattcheeks,Borin sunday
308931,0,2000797261,Mon Jun 01 22:59:34 PDT 2009,NO_QUERY,Jwanni3,My computer just gave me the blue screen
1593443,4,2191942996,Tue Jun 16 06:14:34 PDT 2009,NO_QUERY,July5th,was I with you ? @RubyWuu


In [38]:
df.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [39]:
X=df.drop(columns=['target'],axis=1)
Y=df['target']

In [40]:
print(X)
print(Y)

                 id                          date      flag             user  \
0        1467810369  Mon Apr 06 22:19:45 PDT 2009  NO_QUERY  _TheSpecialOne_   
1        1467810672  Mon Apr 06 22:19:49 PDT 2009  NO_QUERY    scotthamilton   
2        1467810917  Mon Apr 06 22:19:53 PDT 2009  NO_QUERY         mattycus   
3        1467811184  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY          ElleCTF   
4        1467811193  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY           Karoli   
...             ...                           ...       ...              ...   
1599995  2193601966  Tue Jun 16 08:40:49 PDT 2009  NO_QUERY  AmandaMarie1028   
1599996  2193601969  Tue Jun 16 08:40:49 PDT 2009  NO_QUERY      TheWDBoards   
1599997  2193601991  Tue Jun 16 08:40:49 PDT 2009  NO_QUERY           bpbabe   
1599998  2193602064  Tue Jun 16 08:40:49 PDT 2009  NO_QUERY     tinydiamondz   
1599999  2193602129  Tue Jun 16 08:40:50 PDT 2009  NO_QUERY   RyanTrevMorris   

                                       

In [41]:
Y.value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

convert the target '4' to '1'


In [42]:
df['target']=df['target'].astype(str).replace('4', '1',regex= True)

In [43]:
df['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

In [44]:
df['target']=df['target'].astype('int32')

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   target  1600000 non-null  int32 
 1   id      1600000 non-null  int64 
 2   date    1600000 non-null  object
 3   flag    1600000 non-null  object
 4   user    1600000 non-null  object
 5   text    1600000 non-null  object
dtypes: int32(1), int64(1), object(4)
memory usage: 67.1+ MB


In [46]:
port_stem = PorterStemmer()
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if word not in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content 

In [ ]:
 df['text'] = df['text'].apply(stemming) 

In [48]:
df['text']=df['text'].apply(stemming)

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,nationwideclass behav mad see


In [53]:
X=df['text'].values
Y=df['target'].values

In [54]:
print(X)
print(Y)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']
[0 0 0 ... 1 1 1]


In [57]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y , test_size=0.2 ,stratify =Y,random_state =2 )

In [58]:
print(X)

  (0, 531139)	0.41853340623322255
  (0, 485652)	0.18650232234185593
  (0, 469145)	0.2981635147613446
  (0, 452892)	0.41603893025684235
  (0, 422817)	0.32820995182825324
  (0, 197540)	0.14404398117225717
  (0, 175126)	0.14954093240429162
  (0, 108286)	0.1308485829746254
  (0, 107364)	0.2570984091088748
  (0, 90887)	0.15464775678741802
  (0, 71793)	0.37353267903942167
  (0, 64576)	0.27697101648723665
  (0, 33904)	0.23205040088719647
  (1, 492307)	0.3363472184767577
  (1, 492070)	0.2858720996416189
  (1, 474893)	0.19260385801922084
  (1, 461680)	0.3081885537273631
  (1, 410935)	0.248122204888589
  (1, 391097)	0.36311136878699307
  (1, 305666)	0.28235673835888475
  (1, 146672)	0.31522075915645575
  (1, 97384)	0.28899220058167335
  (1, 50970)	0.37297081563086687
  (1, 14528)	0.27918379212736855
  (2, 472107)	0.17104555528508644
  :	:
  (1599996, 186920)	0.20494561586351365
  (1599996, 155773)	0.24109999593798195
  (1599996, 93166)	0.19534599071514144
  (1599996, 90887)	0.1516671930208144
  

In [61]:
model= LogisticRegression(solver='liblinear')

In [62]:
model.fit(X_train, Y_train)

LogisticRegression(solver='liblinear')

In [63]:
X_train_prediction = model.predict(X_train)
training_data_accuracy= accuracy_score(X_train_prediction, Y_train)

In [64]:
print('Accuracy score of the training data :',training_data_accuracy)

Accuracy score of the training data : 0.81042890625


In [65]:
X_test_prediction = model.predict(X_test)
test_data_accuracy= accuracy_score(X_test_prediction, Y_test)

In [66]:
print('Accuracy score of the test data :',test_data_accuracy)

Accuracy score of the test data : 0.77805625
